### aim: 

### date: 

In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [2]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [3]:
import numpy as np
from cmocean import cm
import cartopy as cp
import cartopy.crs as ccrs
import netCDF4 as nc
import matplotlib.pyplot as plt
import xarray as xr
import sys
sys.path.append('/gpfs/home/mep22dku/scratch/SOZONE')
#list of models
sys.path.append('/gpfs/home/mep22dku/scratch/SOZONE/UTILS')
import lom
import utils as ut

%matplotlib inline
import warnings
from datetime import datetime
warnings.filterwarnings('ignore')
import cartopy.feature as cfeature
from importlib import reload
import matplotlib.path as mpath
import glob
import pickle
import pandas as pd
import seawater
import time
plt.rcParams.update({'font.size': 12})
font = {'family' : 'normal',
'weight' : 'normal',
'size'   : 12}

plt.rc('font', **font)

In [4]:
scendict = {
    '1A': {
        'hist_str': 'bc370',
        'fut_str': 'be682',
        'name': 'HIST.OZONE \n LOW TEMP.',
        'name2':'1A: NatlOzone-SSP126',
        'color':'#E8D215',
        'color2':'orange'},
    '1B': {
        'hist_str': 'bc370',
        'fut_str': 'ce417',
        'name': 'HIST. OZONE \n HIGH TEMP.',
        'name2':'1B: NatlOzone-SSP370',
        'color':'#87800A',
        'color2':'orangered'},
    '2A': {
        'hist_str': 'cj198',
        'fut_str': 'cj880',
        'name': 'FIXED OZONE \n LOW TEMP.',
        'name2':'2A: Ozone1950-SSP126',
        'color':'#2DC18E',
        'color2':'mediumseagreen'},
    '2B': {
        'hist_str': 'cj198',
        'fut_str': 'cj881',
        'name': 'FIXED OZONE \n HIGH TEMP.',
        'name2':'2B: Ozone1950-SSP370',
        'color':'#18765C',
        'color2':'green'},
    '3A': {
        'hist_str': 'cj200',
        'fut_str': 'cj484',
        'name': '1990 OZONE \n LOW TEMP.',
        'name2':'3A: Ozone1990-SSP126',
        'color':'#FF462B',
        'color2':'dodgerblue'},
    '3B': {
        'hist_str': 'cj200',
        'fut_str': 'cj504',
        'name': '1990 OZONE \n HIGH TEMP.',
        'name2':'3B: Ozone1990-SSP370',
        'color':'#822722',
        'color2':'mediumblue'}
}


def make_yearfile(td, yr, filename):
    times = pd.date_range(f"{yr}/01/15",f"{yr+1}/01/15",freq='M',closed='left')

    stor_overturning = np.zeros([12,75,332,1])
    
    for i in range(0,12):
        w = xr.open_dataset(td[i])
        stor_overturning[i,:,:,:] = w.zomsfglo.values

    data_vars = {'zomsfglo':(['time_counter', 'depthw', 'y', 'x'], stor_overturning,
    {'units': 'Sv',
    'long_name':'depth-coordinates overturning'}),
    }

    coords = {'time_counter': (['time_counter'], times),
     #       'time_centered': (['time_centered'], times),
    'nav_lat': (['y','x'], w.nav_lat),
    'nav_lon': (['y','x'], w.nav_lon),
    'depthw': (['depthw'], w.depthw)}

    attrs = {'made in':'overturning/makeyearfiles_overturning.ipynb',
    'desc': 'concatenate monthly overturning output into yearly and give good dates for later processing'
    }

    savenam = filename
    ds = xr.Dataset(data_vars=data_vars,
    coords=coords,
    attrs=attrs)
    ds.to_netcdf(savenam)

def make_yearfiles_ukesm(yrst, yren, tscen):
    print(f'SCENARIO {tscen}')
    dslist = []

    for y in range(yrst,yren):
        if ((y<1990) & ((tscen == '3A') | (tscen == '3B'))):
            tstr = scendict['1A']['hist_str']
        elif y<2015:
            tstr = scendict[tscen]['hist_str']

        else:
            tstr = scendict[tscen]['fut_str']

        try:

            td = glob.glob(f'/gpfs/data/greenocean/software/resources/MEDUSA/ukesm_allscen_moc/nemo_{tstr}o_1m_{y}*_moc.nc')
            if (len(td) != 12):
                print(f'SCENARIO {tscen}, {y}, something missing')
            else:
                td = sorted(td)
                filename = f'/gpfs/data/greenocean/software/resources/MEDUSA/ukesm_allscen_moc_byyear//nemo_{tstr}o_1y_{y}_moc.nc'
                try:
                    make_yearfile(td, y, filename)
                    #print(f'SCENARIO {tscen}, {y}, made yearly file')
                except:
                    print(f'SCENARIO {tscen}, {y}, yearly file  not made but all monthly files present')

            #dslist.append(td[0])
        except:
            pass
            #print(f'FAIL {tscen},{tstr}, {y}')
    return td

# td =make_yearfiles_ukesm(1950, 2100, '1A')
# td =make_yearfiles_ukesm(1950, 2100, '1B')
# td =make_yearfiles_ukesm(1950, 2100, '2A')
# td =make_yearfiles_ukesm(1950, 2100, '2B')
# td =make_yearfiles_ukesm(1950, 2100, '3A')
# td =make_yearfiles_ukesm(1950, 2100, '3B')

In [6]:

def make_yearfile_sig(td, yr, filename):
    times = pd.date_range(f"{yr}/01/15",f"{yr+1}/01/15",freq='M',closed='left')

    dmoc0 = np.zeros([12,52,332])
    dmoc1000 = np.zeros([12,88,332])
    dmoc2000 = np.zeros([12,158,332])
    
    for i in range(0,12):
        w = xr.open_dataset(td[i])
        dmoc0[i,:,:] = w.dmoc0.values
        dmoc1000[i,:,:] = w.dmoc1000.values
        dmoc2000[i,:,:] = w.dmoc2000.values

    latV = w.latV.values
    sigma0 = w.sigma0.values
    sigma1000 = w.sigma1000.values
    sigma2000 = w.sigma2000.values
    
    data_vars = {'dmoc0':(['time_counter', 'sigma0', 'latV'], dmoc0,
    {'units': 'Sv',
    'long_name':'sigma-coordinates overturning, ref pressure 0 dbar'}),

                 'dmoc1000':(['time_counter', 'sigma1000', 'latV'], dmoc1000,
    {'units': 'Sv',
    'long_name':'sigma-coordinates overturning, ref pressure 1000 dbar'}),

                 'dmoc2000':(['time_counter', 'sigma2000', 'latV'], dmoc2000,
    {'units': 'Sv',
    'long_name':'sigma-coordinates overturning, ref pressure 2000 dbar'}),
    }

    coords = {'time_counter': (['time_counter'], times),
 #       'time_centered': (['time_centered'], times),
    'latV': (['latV'],latV),
    'sigma0': (['sigma0'], sigma0),
    'sigma1000': (['sigma1000'], sigma1000),
    'sigma2000': (['sigma2000'], sigma2000),
             }

    attrs = {'made in':'overturning/overturning/sigma_overturning.ipynb',
    'desc': 'concatenate monthly overturning output into yearly and give good dates for later processing'
    }
    
    savenam = filename
    ds = xr.Dataset(data_vars=data_vars,
    coords=coords,
    attrs=attrs)
    ds.to_netcdf(savenam)

    savenam = filename
    ds = xr.Dataset(data_vars=data_vars,
    coords=coords,
    attrs=attrs)
    ds.to_netcdf(savenam)
    #return ds


In [7]:
def make_yearfiles_ukesm_sig(yrst, yren, tscen):
    print(f'SCENARIO {tscen}')
    dslist = []

    for y in range(yrst,yren):
        if ((y<1990) & ((tscen == '3A') | (tscen == '3B'))):
            tstr = scendict['1A']['hist_str']
        elif y<2015:
            tstr = scendict[tscen]['hist_str']

        else:
            tstr = scendict[tscen]['fut_str']

        try:

            td = glob.glob(f'/gpfs/data/greenocean/software/resources/MEDUSA/ukesm_allscen_mocsig//nemo_{tstr}o_1m_{y}*_mocsig.nc')
            if (len(td) != 12):
                print(f'SCENARIO {tscen}, {y}, something missing')
            else:
                td = sorted(td)
                filename = f'/gpfs/data/greenocean/software/resources/MEDUSA/ukesm_allscen_moc_byyear//nemo_{tstr}o_1y_{y}_mocsig.nc'
                try:
                    make_yearfile_sig(td, y, filename)
                    #print(f'SCENARIO {tscen}, {y}, made yearly file')
                except:
                    print(f'SCENARIO {tscen}, {y}, yearly file  not made but all monthly files present')

            #dslist.append(td[0])
        except:
            pass
            #print(f'FAIL {tscen},{tstr}, {y}')
    return td

In [8]:
# td =make_yearfiles_ukesm_sig(1950, 2100, '1A')
# td =make_yearfiles_ukesm_sig(1950, 2100, '1B')
# td =make_yearfiles_ukesm_sig(1950, 2100, '2A')
# td =make_yearfiles_ukesm_sig(1950, 2100, '2B')
# td =make_yearfiles_ukesm_sig(1950, 2100, '3A')
# td =make_yearfiles_ukesm_sig(1950, 2100, '3B')

In [9]:
def make_yearlist(yrst, yrend, tscen, \
                  tdir = '/gpfs/home/mep22dku/scratch/SOZONE/windAnalyis/oceanFields/max_OT/'):
    yrs = np.arange(yrst,yrend+1,1)
    ylist = []
    for i in range(0,len(yrs)):
        yr = yrs[i]
        ty = f'{tdir}/seas_OT_{yr}_{tscen}.nc'
        t2 = glob.glob(ty)
        #print(t2)
        ylist.append(t2[0])
    return ylist

ot_ukesm_1A = xr.open_mfdataset(make_yearlist(1950,1951,'1A'))

o
# ot_ukesm_2A = xr.open_mfdataset(make_yearlist(1950,2099,'2A'))
# ot_ukesm_3A = xr.open_mfdataset(make_yearlist(1950,2099,'3A'))
# ot_ukesm_1B = xr.open_mfdataset(make_yearlist(1950,2099,'1B'))
# ot_ukesm_2B = xr.open_mfdataset(make_yearlist(1950,2099,'2B'))
# ot_ukesm_3B = xr.open_mfdataset(make_yearlist(1950,2099,'3B'))

KeyboardInterrupt: 

In [10]:
w = xr.open_dataset('/gpfs/home/mep22dku/scratch/SOZONE/windAnalyis/oceanFields/max_OT/seas_OT_1950_1A.nc')
w

<xarray.Dataset>
Dimensions:          (time_counter: 4)
Coordinates:
  * time_counter     (time_counter) datetime64[ns] 1950-01-01 ... 1950-10-01
Data variables:
    max_OT           (time_counter) float64 ...
    lat_OT           (time_counter) float64 ...
    sigma2000_OT     (time_counter) float64 ...
    max_OT_yr        (time_counter) float64 ...
    lat_OT_yr        (time_counter) float64 ...
    sigma2000_OT_yr  (time_counter) float64 ...
Attributes:
    made in:  SOZONE/windAnalyis/oceanFields/OT.ipynb
    desc:     where and when does maximum overturning occur